In [ ]:
%pip install languagemodels
import languagemodels as lm

In [ ]:
!pip install --upgrade openai
!pip install xlrd
!pip install -U sentence-transformers
!pip install tqdm

In [ ]:
import numpy as np
dataset = np.load('/content/SP-train.npy', allow_pickle=True)
answers = []
questions = []
choices = []
for i in range(len(dataset)):
  answers.append(dataset[i]["answer"])
  questions.append(dataset[i]["question"])
  choices.append(dataset[i]["choice_list"])

import math
from numpy.linalg import norm
import openpyxl
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')
wb = openpyxl.load_workbook('/content/EncodedQuestions.xlsx')
ws = wb.active

def findRiddle(Riddle):
  question_emb = model.encode(Riddle)
  A= question_emb
  norm_A = norm(A)
  highest = 0
  for i in range(1,ws.max_row):
    B = [float(x) for x in ws["C"+str(i)].value.split(",")]
    sim = np.dot(A,B)/(norm_A*norm(B))
    if sim > highest:
        highest = sim
        nearest = i
  return [ws["A"+str(nearest)].value, ws["B"+str(nearest)].value]


In [ ]:
!sudo apt-get install libcublas-dev

In [ ]:
from difflib import SequenceMatcher
import json
from tqdm import tqdm
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()


models = ["neural-chat-7b-v3-1", "flan-alpaca-xl", "flan-alpaca-gpt4-xl", "flan-t5-xl", "fastchat-t5-3b-v1.0", "LaMini-Flan-T5-783M", "flan-t5-large", "LaMini-Flan-T5-248M", "flan-alpaca-base", "flan-t5-base", "dialogstudio-t5-base-v1.0", "LaMini-Flan-T5-77M", "flan-t5-small", "phi-1_5", "LaMini-GPT-774M", "LaMini-GPT-124M", "TinyLlama-1.1B-Chat-v0.3"]
for modelname in models:
  correct = 0
  count = 0
  lm.config["instruct_model"] = modelname
  for x in tqdm(range(0,len(questions))):
      closeRiddle = findRiddle(questions[x])
      #closeRiddle = farRiddle(questions[x])
      demonstration = "Please pick the best choice for the sentence puzzler brain teaser (Sentence-type brain teaser where the puzzle defying commonsense is centered on sentence snippets). Each brain teaser has only one possible solution including the choice none of above, answer should only provide the choice:"
      prompt = demonstration + "Question:" + questions[x]+ " \n Choice:" + "\n (A) " + choices[x][0] + "\n (B) " + choices[x][1] + "\n (C) " + choices[x][2] + "\n (D) " +choices[x][3] + "\n Example Questions with corresponding example answer:" +"\n" +closeRiddle[0] + "\n The answer would be" + closeRiddle[1]
      pred =lm.do(prompt)

      try:
        pred = json.loads(pred)['response']
      except:
        pred = pred
      answer = choices[x][0]
      val = similar(choices[x][0], pred)
      if val < similar(choices[x][1], pred):
        answer = choices[x][1]
        val = similar(choices[x][1],pred)
      if val < similar(choices[x][2], pred):
        answer = choices[x][2]
        val = similar(choices[x][2],pred)
      if val < similar(choices[x][3], pred):
        answer = choices[x][3]
        val = similar(choices[x][3],pred)
      #Calculate Accuracy
      #print(answer)
      if answer  == answers[x]:
        correct += 1
        #print("Correct" + answer)
        #print(answers[x])
      count += 1

  print(modelname, correct, count, correct/count)



In [ ]:
from difflib import SequenceMatcher
import json
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

correct = 0
count = 0
lm.store_doc(lm.get_wiki("Riddle"))
for x in range(0,len(questions)):
    closeRiddle = findRiddle(questions[x])
    #closeRiddle = farRiddle(questions[x])
    demonstration = "Please pick the best choice for the sentence puzzler brain teaser (Sentence-type brain teaser where the puzzle defying commonsense is centered on sentence snippets). Each brain teaser has only one possible solution including the choice none of above, answer should only provide the choice:"
    prompt = demonstration + "Question:" + questions[x]+ " \n Choice:" + "\n (A) " + choices[x][0] + "\n (B) " + choices[x][1] + "\n (C) " + choices[x][2] + "\n (D) " +choices[x][3] + "\n Example Questions with corresponding example answer:" +"\n" +closeRiddle[0] + "\n The answer would be" + closeRiddle[1]
    pred =lm.load_doc(prompt)
    try:
      pred = json.loads(pred)['response']
    except:
      pred = pred
    answer = choices[x][0]
    val = similar(choices[x][0], pred)
    if val < similar(choices[x][1], pred):
      answer = choices[x][1]
      val = similar(choices[x][1],pred)
    if val < similar(choices[x][2], pred):
      answer = choices[x][2]
      val = similar(choices[x][2],pred)
    if val < similar(choices[x][3], pred):
      answer = choices[x][3]
      val = similar(choices[x][3],pred)
    #Calculate Accuracy
    print(answer)
    if answer  == answers[x]:
      correct += 1
      print("Correct" + answer)
      print(answers[x])
    else:
      print("Incorrect:" + pred)
      print(answers[x])

    count += 1

print(correct, count, correct/count)

